In [1]:
import requests, json

In [2]:
# Use cached results from previous user searches (Skyscanner API on RapidAPI)

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "a25c140655mshcc3de02bc48e78fp1d8ae5jsn716b89dc608a"
    }

## Start with one-way tickets and find cheapest option

* Two origins: Madrid and Zurich 
* Three city destinations (consider all airports in the three cities): London, Paris, Rome
* date: single date, & one-way

**goal**: chose cheapest city to fly to from Madrid and Zurich, i.e. city that will result in min total cost

note TODO: create a database with city and/or airport names and corresponding IDs for skyscanner so can use user queries (in the form of city for ex) and translate it to ID keys

note TODO: make sure all future origins also takes into account multiple airports, in this case our two origin airports only have one, whereas Paris has a possibility of 4.

In [59]:
# from Spain (ES-sky) - MAD (MAD-sky) & Switzerland (CH-sky) - Zurich (ZURI-sky)
origin_list = ["MAD-sky", "ZURI-sky"]

# where we are doing search from 
origin_country = "ES"

# list of possible destinations 
destination_list = ["LOND-sky", "PARI-sky", "ROME-sky"]

currency = "EUR"
locale = "en-US"
date = "2021-02-01" #yyyy-mm-dd

#for origin in origin_list:
    #print(f'For origin {origin}:')
    #print()
    
    #for destination in destination_list:
        #myurl = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/" + originCountry + "/" + currency + "/" + locale + "/"  + destination + "/" + origin + "/"+ date
        #response = requests.request("GET", myurl, headers=headers)

        #print(f'For destination {destination}:')
        #print(response.text)
        #print()
        #print('--------')
        #print()
    
    #print('########')
    #print()

In [30]:
# match airport IDs to names of origin and destination
# add to options list with 

airports = {}
options = []
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/"

for origin in origin_list:
    for destination in destination_list:
        myurl = url + origin_country + "/" + currency + "/" + locale + "/"  + origin + "/" + destination + "/"+ date
        response = requests.request("GET", myurl, headers=headers)
        temp = json.loads(response.text)
        
        if("Quotes" in temp):
            for places in temp["Places"]:
                airports[places["PlaceId"]] = places["Name"] 
                
            for quotes in temp["Quotes"]:
                print("-----------")
                ori = quotes["OutboundLeg"]["OriginId"]
                dest = quotes["OutboundLeg"]["DestinationId"]
                price = quotes["MinPrice"]

                print(f"Journey:  {airports[ori]}  --> {airports[dest]}")
                print(f"Price: {price} EUR")
                
                options.append({"origin": airports[ori], "destination": airports[dest], "price": price})

-----------
Journey:  Madrid  --> London Stansted
Price: 31 EUR
-----------
Journey:  Madrid  --> London Gatwick
Price: 31 EUR
-----------
Journey:  Madrid  --> Paris Orly
Price: 36 EUR
-----------
Journey:  Madrid  --> Rome Ciampino
Price: 22 EUR
-----------
Journey:  Zurich  --> London Heathrow
Price: 64 EUR
-----------
Journey:  Zurich  --> London Heathrow
Price: 101 EUR
-----------
Journey:  Zurich  --> Paris Orly
Price: 74 EUR
-----------
Journey:  Zurich  --> Paris Charles de Gaulle
Price: 105 EUR
-----------
Journey:  Zurich  --> Rome Fiumicino
Price: 67 EUR
-----------
Journey:  Zurich  --> Rome Fiumicino
Price: 81 EUR


In [31]:
options

[{'origin': 'Madrid', 'destination': 'London Stansted', 'price': 31},
 {'origin': 'Madrid', 'destination': 'London Gatwick', 'price': 31},
 {'origin': 'Madrid', 'destination': 'Paris Orly', 'price': 36},
 {'origin': 'Madrid', 'destination': 'Rome Ciampino', 'price': 22},
 {'origin': 'Zurich', 'destination': 'London Heathrow', 'price': 64},
 {'origin': 'Zurich', 'destination': 'London Heathrow', 'price': 101},
 {'origin': 'Zurich', 'destination': 'Paris Orly', 'price': 74},
 {'origin': 'Zurich', 'destination': 'Paris Charles de Gaulle', 'price': 105},
 {'origin': 'Zurich', 'destination': 'Rome Fiumicino', 'price': 67},
 {'origin': 'Zurich', 'destination': 'Rome Fiumicino', 'price': 81}]

In [32]:
# self join -> there might be a more streamlined way of doing this (w pandas? look up)
dd_options = [options, options]

In [60]:
st = "Paris Charles de Gaulle"
st.split(" ", 1)

['Paris', 'Charles de Gaulle']

In [76]:
# Find common destination (city-wise) and calculate total price
price = []
dest_1_list = []
dest_2_list = []

for option_1 in dd_options[0]:
    
    #to avoid repeats, just read the first of the two origins
    if option_1["origin"] == "Madrid":
        
        for option_2 in dd_options[1]:
            # parse destination to get city (ex London) so that compare common cities, not just common airports
            city_1, airport_1 = option_1["destination"].split(" ", 1)
            city_2, airport_2 = option_2["destination"].split(" ", 1)

            if (option_1["origin"] != option_2["origin"]) & (city_1 == city_2) :

                ori_1 = option_1["origin"]
                ori_2 = option_2["origin"]
                dest_1 = option_1["destination"] 
                dest_2 = option_2["destination"]

                sum_price = option_1["price"] + option_2["price"]
                price.append(sum_price)
                dest_1_list.append(dest_1)
                dest_2_list.append(dest_2)

                print(f"Common destination:")
                print(f"Journey: {ori_1} -> {dest_1}")
                print(f"Journey: {ori_2} -> {dest_2}")
                print(f"Total price: {sum_price}")
                print("-----------")


Common destination:
Journey: Madrid -> London Stansted
Journey: Zurich -> London Heathrow
Total price: 95
-----------
Common destination:
Journey: Madrid -> London Stansted
Journey: Zurich -> London Heathrow
Total price: 132
-----------
Common destination:
Journey: Madrid -> London Gatwick
Journey: Zurich -> London Heathrow
Total price: 95
-----------
Common destination:
Journey: Madrid -> London Gatwick
Journey: Zurich -> London Heathrow
Total price: 132
-----------
Common destination:
Journey: Madrid -> Paris Orly
Journey: Zurich -> Paris Orly
Total price: 110
-----------
Common destination:
Journey: Madrid -> Paris Orly
Journey: Zurich -> Paris Charles de Gaulle
Total price: 141
-----------
Common destination:
Journey: Madrid -> Rome Ciampino
Journey: Zurich -> Rome Fiumicino
Total price: 89
-----------
Common destination:
Journey: Madrid -> Rome Ciampino
Journey: Zurich -> Rome Fiumicino
Total price: 103
-----------


In [104]:
# find cheapest destination (city-wise)
city_price = {}
airport_1_dict = {}
airport_2_dict = {}

for i, (d1, d2) in enumerate(zip(dest_1_list, dest_2_list)):
    # from mad
    city_1, airport_1 = d1.split(" ", 1)
    
    # from zuri
    city_2, airport_2 = d2.split(" ", 1)
    
    # if two destination cities are the same, then add price to corresponding dict entry in city_price
    if city_1 == city_2:
        if city_1 not in city_price:
            city_price[city_1] = [price[i]]
        else:
            city_price[city_1].append(price[i])  
            
        if city_1 not in airport_1_dict:
            airport_1_dict[city_1] = [airport_1]
        else:
            airport_1_dict[city_1].append(airport_1) 

        if city_2 not in airport_2_dict:
            airport_2_dict[city_2] = [airport_2]
        else:
            airport_2_dict[city_2].append(airport_2) 

In [92]:
city_price

{'London': [95, 132, 95, 132], 'Paris': [110, 141], 'Rome': [89, 103]}

In [105]:
airport_1_dict

{'London': ['Stansted', 'Stansted', 'Gatwick', 'Gatwick'],
 'Paris': ['Orly', 'Orly'],
 'Rome': ['Ciampino', 'Ciampino']}

In [106]:
airport_2_dict

{'London': ['Heathrow', 'Heathrow', 'Heathrow', 'Heathrow'],
 'Paris': ['Orly', 'Charles de Gaulle'],
 'Rome': ['Fiumicino', 'Fiumicino']}

In [109]:
cheapest_city_price = {}

for city in city_price:
    i = city_price[city].index(min(city_price[city]))
    cheapest_city_price[city] = min(city_price[city])
    cheapest_a1 = airport_1_dict[city][i]
    cheapest_a2 = airport_2_dict[city][i]

In [95]:
cheapest_city_price

{'London': 95, 'Paris': 110, 'Rome': 89}

In [110]:
cheapest_a1 

'Ciampino'

In [111]:
cheapest_a2

'Fiumicino'

In [113]:
# Find cheapest destination out of the 3 city options 

cheapest_dest = (min(cheapest_city_price, key=cheapest_city_price.get))
cheapest_price = min(cheapest_city_price.values())

print(f"Cheapest destination:")
print('------')
print(f"Journey: Madrid -> {cheapest_dest}, Airport: {cheapest_a1}")
print(f"Journey: Zurich -> {cheapest_dest}, Airport: {cheapest_a2}")
print(f"Total price: {cheapest_price} EUR")
print('------')


Cheapest destination:
------
Journey: Madrid -> Rome, Airport: Ciampino
Journey: Zurich -> Rome, Airport: Fiumicino
Total price: 89 EUR
------


additionally: recover the breakdown of price, i.e. Madrid-Ciampino and Zurich-Fiumicino